In [5]:
from selenium import webdriver # For Selenium webdriver
from selenium.webdriver.common.by import By #For Tag
from selenium.webdriver.chrome.options import Options # From Selenium we use Options replace to ChormeOptions
from selenium.webdriver.common.proxy import Proxy, ProxyType # To use proxy server
from selenium.common.exceptions import StaleElementReferenceException # To prevent errors
from selenium.webdriver.support.ui import WebDriverWait # WebDriver wait function to wait loading DOM
from selenium.webdriver.support import expected_conditions as EC
from datetime import datetime, timedelta 
from openpyxl.styles import Alignment, Font, colors, Border, Side, PatternFill

import openpyxl
import random
import pickle
import time


# Create an Excel file and setting (bold head, columns dimension)

excel_file = openpyxl.Workbook()
excel_sheet = excel_file.active
excel_sheet.title = 'naver flight lowest price'

excel_sheet.column_dimensions['A'].width = 20
excel_sheet.column_dimensions['A'].width = 15
excel_sheet.column_dimensions['A'].width = 15
excel_sheet.column_dimensions['A'].width = 15
excel_sheet.column_dimensions['A'].width = 15
excel_sheet.column_dimensions['A'].width = 20

header_bold_font = Font(bold = True, size = 15)
for cell in excel_sheet[1]:
    cell.font = header_bold_font

# Random User-Agent setup to avoid web crawling block
user_agents = [
    "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/91.0.4472.124 Safari/537.36",
    "Mozilla/5.0 (Macintosh; Intel Mac OS X 10_15_7) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/89.0.4389.90 Safari/537.36",
    "Mozilla/5.0 (iPhone; CPU iPhone OS 14_6 like Mac OS X) AppleWebKit/605.1.15 (KHTML, like Gecko) Version/14.1.1 Mobile/15E148 Safari/604.1"
]

# Proxy setup
proxy_list = [
    "23.247.136.252:80",
    "43.134.14.33:8899",
    "47.245.91.224:10001"
]

proxy = Proxy() # Proxy class object
proxy.proxy_type = ProxyType.MANUAL # Set the proxy manual setup
proxy.http_proxy = random.choice(proxy_list)


#Chrome Option setup
option = Options()
option.add_argument(f"user-agent = {random.choice(user_agents)}")
option.add_argument("--disable-blink-features=AutomationControlled") #Blink engine feature off


# Date input function     # end_date를 계산하기 위해서 start_date를 datetime으로 바꿨다가 다시 변환하는 작업
def get_date_duration_input():
    start_date = input('Enter the start date (YYYY.MM.DD): ')
    duration = int(input('Enter the duration in days: '))

    start_date_obj = datetime.strptime(start_date, "%Y.%m.%d")
    end_date_obj = start_date_obj + timedelta(days = duration)

    start_date = start_date_obj.strftime("%Y.%m.%d")    
    end_date = end_date_obj.strftime("%Y.%m.%d")
    
    return start_date, end_date

# YYYY.MM.DD select function
def select_dates(driver, current_start_date, current_end_date):
    start_year_month, start_day = current_start_date[:8], current_start_date[8:]
    end_year_month, end_day = current_end_date[:8], current_end_date[8:]

    def scroll_year_month(target_year_month):
        scroll_container = driver.find_element(By.CSS_SELECTOR, 'div.container_content__a1aRX.container_as_calendar__ztByS')
        scroll_position = 0
    
        while True:
            year_month_elements = driver.find_elements(By.CSS_SELECTOR, 'div.sc-dAlyuH.cKxEnD')
            current_year_month = [year_month_element.text.strip() for year_month_element in year_month_elements]
    
            if target_year_month in current_year_month:
                break

            scroll_position += 300
            driver.execute_script("arguments[0].scrollTop += 300", scroll_container)
            time.sleep(0.5)
    
    def click_date(target_year_month, target_day):
        while True:
            try:
                month_divs = driver.find_elements(By.CSS_SELECTOR, 'div.sc-kpDqfm.ljuuWQ.month')
                for month_div in month_divs:
                    title_element = month_div.find_element(By.CSS_SELECTOR, 'div.sc-dAlyuH.cKxEnD')
                    if title_element.text.strip() == target_year_month:
                        table_element = month_div.find_element(By.CSS_SELECTOR, 'table')
                        
                        day_elements = month_div.find_elements(By.CSS_SELECTOR, 'b.sc-cwHptR.hFwLsU.num')
                        for day_element in day_elements:
                            if day_element.text.strip() == target_day:
                                day_element.click()
                                return
                        
            except StaleElementReferenceException:
                print("Stale element encountered, retrying...")
                time.sleep(0.5) 

         



    scroll_year_month(start_year_month)
    click_date(start_year_month, start_day)
    scroll_year_month(end_year_month)
    click_date(end_year_month, end_day)
    
# Execute Webdriver
if __name__ == "__main__":
    driver = webdriver.Chrome(options = option)
    driver.get('https://flight.naver.com/')
    time.sleep(3)
    
     # To prepare to save in an EXCEL file
    
    excel_sheet.append(['Airline', '(D)Departure time', 'Arrival time', '(R)departure time', 'Arrival time','Price'])

    start_date, end_date = get_date_duration_input()
    repeat_days = 1
    
  
    try:
        for i in range(repeat_days):
            current_start_date = (datetime.strptime(start_date, "%Y.%m.%d") + timedelta(days = i)).strftime("%Y.%m.%d")
            current_end_date = (datetime.strptime(end_date, "%Y.%m.%d") + timedelta(days = i)).strftime("%Y.%m.%d")
            print(f'processing data for{current_start_date} - {current_end_date}')
            
            
            # turn around button
            element = driver.find_element(By.CSS_SELECTOR, 'button[aria-selected = "true"]')
            element.click()
            time.sleep(1)
            
            # Depart button
            element = driver.find_element(By.CSS_SELECTOR, "button.tabContent_route__EXyDz.select_City__mKbzk.start")
            element.click()
            time.sleep(1)
        
            # From Incheon 
            element = driver.find_element(By.XPATH, '//button[contains(text(), "인천")]')
            element.click()
            time.sleep(1)
        
            # Arrive button
            element = driver.find_element(By.CSS_SELECTOR, "button.tabContent_route__EXyDz.select_City__mKbzk.end")
            element.click()
            time.sleep(1)
        
            # To Singapore
            element = driver.find_element(By.CSS_SELECTOR, "input[placeholder = '국가, 도시, 공항명 검색']")
            element.clear()
            element.send_keys('싱가포르')
            time.sleep(1)
            element = driver.find_element(By.CSS_SELECTOR, 'li[data-event-area-code = "arr.autoairport"]')
            element.click()
            time.sleep(2)
    
            # Depart, Return button
            elements = driver.find_elements(By.CSS_SELECTOR, 'div.tabContent_options__4pr6W')
            elements[0].click()
            time.sleep(1)
            
            # Direct flight option button
            direct_button = driver.find_element(By.CSS_SELECTOR, 'button.tabContent_option___mYJO.tabContent_as_direct__dOk9T.select_Direct__VsxJp')
            is_checked = direct_button.get_attribute("aria-checked") == 'true'
            if not is_checked:    
                direct_button.click()
            time.sleep(1)
    
            # Before select date button
            departure_return_date = driver.find_elements(By.CSS_SELECTOR, 'button.tabContent_option___mYJO.select_Date__Potbp')
            departure_return_date[0].click()
            time.sleep(1)

            # Reset selected date
            reset_date_button = driver.find_element(By.CSS_SELECTOR, 'button.calendar_clear__thTxk')
            if i != 0:
                reset_date_button.click()
    
            
            # Choose the date function
            select_dates(driver, current_start_date, current_end_date)
    
            time.sleep(2)
    
            element = driver.find_element(By.CSS_SELECTOR, 'button.searchBox_search__dgK4Z')
            element.click()
    
            time.sleep(10)

            
            #lowest price scrappping
            lowest_element = driver.find_element(By.XPATH, '//i[text() = "최저가"]/ancestor::div[contains(@class, "concurrent_ConcurrentItemContainer__NDJda")]')
                
            
            airline_name = lowest_element.find_element(By.CSS_SELECTOR, 'span.airline_text__WWkbY').text.strip()
            
            route_elements = lowest_element.find_elements(By.CSS_SELECTOR,'span.route_airport__tBD9o')
            
            d_departure_time = route_elements[0].text.strip()
            d_arrival_time = route_elements[1].text.strip()
            r_departure_time = route_elements[2].text.strip()
            r_arrival_time = route_elements[3].text.strip()
            
            price = lowest_element.find_element(By.CSS_SELECTOR,'b.item_usual__wg002').text.strip()

            excel_sheet.append([airline_name,d_departure_time,d_arrival_time,r_departure_time,r_arrival_time, price])

            # home button click 
            time.sleep(7)
            element = driver.find_element(By.CSS_SELECTOR, 'span.Header_nav_item_text__zBZUG')
            element.click()


    
    except Exception as error:
        print("Error:", error)
        
    finally:
        driver.quit()

# Save as Excel file and close
excel_file.save('naver flight lowest price crawling.xlsx')
excel_file.close()




Enter the start date (YYYY.MM.DD):  2025.01.10
Enter the duration in days:  2


processing data for2025.01.10 - 2025.01.12
Error: Message: no such element: Unable to locate element: {"method":"xpath","selector":"//i[text() = "최저가"]/ancestor::div[contains(@class, "concurrent_ConcurrentItemContainer__NDJda")]"}
  (Session info: chrome=131.0.6778.205); For documentation on this error, please visit: https://www.selenium.dev/documentation/webdriver/troubleshooting/errors#no-such-element-exception
Stacktrace:
0   chromedriver                        0x0000000102b8e138 cxxbridge1$str$ptr + 3653888
1   chromedriver                        0x0000000102b86988 cxxbridge1$str$ptr + 3623248
2   chromedriver                        0x00000001025ec968 cxxbridge1$string$len + 89228
3   chromedriver                        0x0000000102630d4c cxxbridge1$string$len + 368752
4   chromedriver                        0x000000010266a4f0 cxxbridge1$string$len + 604180
5   chromedriver                        0x0000000102625564 cxxbridge1$string$len + 321672
6   chromedriver                    

### Proxy server test whether it works or not

In [16]:
# Proxy server test


from selenium import webdriver
from selenium.webdriver.chrome.options import Options


proxy = '23.247.136.252:80'

options = Options()
options.add_argument(f"--proxy-server = {proxy}")
try:
    driver = webdriver.Chrome(options = options)
    driver.get('https://www.skyscanner.com.sg/')
    print('Working well')

except Exception as e:
    print('not working', e)

finally:
    driver.quit()




Working well
